## Clean Code

In [111]:
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from scipy import stats
from sklearn import tree
import os
import xlrd
import pandas as pd
import seaborn
%matplotlib inline
import csv
from csv import reader
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from matplotlib.lines import Line2D
from sklearn.preprocessing import StandardScaler
from scipy.stats import pearsonr
import pickle
RF_SEED = 30

In [112]:
# Read the csv file into a pandas DataFrame

Property = pd.read_csv('Resources/Property.csv')
Property

C:\Users\coset\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3071: DtypeWarning: Columns (5) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


,Parcel_ID,Parcel_Acreage,Property_Street_Address,Property_City,Property_State,Property_Zip,Parcel_Vacancy,Conveyance_Date,Property_Sales_Price,Seller_City,...,Buyer_ZIP,Primary_Residence,Assessed_Value_Land,Assessed_Value_Improvement,Total_Assessed_Value,Property_Class_Code,Class_Code_General,Class_Code_Translated,County_ID,County_Name
0,4574368,0.169,3909 MARIAH DRIVE ...,NEW ALBANY,IN,41750,N,6/2/2020,179900.0,NEW ALBANY,...,47150,Y,43800.0,138500.0,182300.0,510.0,500,Residential,22,Floyd
1,4515880,0.134,4110 E 10TH STREET,INDIAAPOLIS,IN,42601,N,10/19/2020,48000.0,Petaluma,...,46074,N,5000.0,88500.0,93500.0,510.0,500,Residential,49,Marion
2,4454504,0.172,5404 OAK HARBOR CT,INDIANAPOLIS,IN,42637,N,6/18/2020,195000.0,Indianapolis,...,46237,Y,30000.0,158000.0,188000.0,520.0,500,Residential,49,Marion
3,4421719,NaN,6516 EMERALD HILL COURT #310,INDIANAPOLIS,IN,42637,N,5/1/2020,120000.0,Franklin,...,42637,Y,10400.0,77900.0,88300.0,550.0,500,Residential,49,Marion
4,4531964,2.155,6335 Intech Commons Drive,Indianapolis,IN,42678,N,12/16/2020,1785000.0,MIAMI BEACH,...,75093,N,845000.0,957900.0,1802900.0,425.0,400,Commercial,49,Marion
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
184184,4666351,4000.000,301 S COBB St ...,PINE VILLAGE,IN,47975,N,5/19/2020,90000.0,WILLIAMSPORT,...,47975,Y,13200.0,51300.0,64500.0,510.0,500,Residential,86,Warren
184185,4666542,2000.000,405 E Fifth St ...,West Lebanon,IN,47991,N,11/4/2020,78000.0,LAFAYETTE,...,47991,Y,5600.0,41100.0,46700.0,510.0,500,Residential,86,Warren
184186,4666463,2700.000,120 W Third ...,Williamsport,IN,47993,N,7/20/2020,11250.0,WEST LEBANON,...,47906,N,8600.0,58600.0,67200.0,511.0,500,Residential,86,Warren
184187,4666265,1564.000,206 Sycamore Dr ...,Williamsport,IN,47993,N,1/31/2020,200000.0,LAFAYETTE,...,47993,Y,7500.0,1700.0,9200.0,599.0,500,Residential,86,Warren


In [113]:
len(Property)

184189

In [114]:
#Drop NA's from DF
pd.DataFrame(Property).dropna()
Property.head()
Property_df = Property[~Property.isin([np.nan, np.inf, -np.inf]).any(1)]
len(Property_df)

180360

In [115]:
#Select 5 Columns of DF
#Property_df=Property_df[['Property_Sales_Price','Total_Assessed_Value','Parcel_Acreage','Parcel_Vacancy','Class_Code_Translated']]
Property_df=Property_df[['Property_Sales_Price','Class_Code_Translated','Total_Assessed_Value','Parcel_Acreage','Parcel_Vacancy', 'Property_Zip' ]]
Property_df.head()

,Property_Sales_Price,Class_Code_Translated,Total_Assessed_Value,Parcel_Acreage,Parcel_Vacancy,Property_Zip
0,179900.0,Residential,182300.0,0.169,N,41750
1,48000.0,Residential,93500.0,0.134,N,42601
2,195000.0,Residential,188000.0,0.172,N,42637
4,1785000.0,Commercial,1802900.0,2.155,N,42678
5,257575.0,Residential,210900.0,0.201,N,45228


In [116]:
num_col = ['Property_Sales_Price','Total_Assessed_Value', 'Parcel_Acreage']

In [117]:

# outliers
print(Property_df[(np.abs(stats.zscore(Property_df[num_col])) > 3)].shape)
# good data
print(Property_df[(np.abs(stats.zscore(Property_df[num_col])) < 3).all(axis=1)].shape)

(646, 6)
(179715, 6)


In [118]:
#Git Rid of Outliers
Property_df = Property_df[(np.abs(stats.zscore(Property_df[num_col])) < 3).all(axis=1)].reset_index()
Property_df


,index,Property_Sales_Price,Class_Code_Translated,Total_Assessed_Value,Parcel_Acreage,Parcel_Vacancy,Property_Zip
0,0,179900.0,Residential,182300.0,0.169,N,41750
1,1,48000.0,Residential,93500.0,0.134,N,42601
2,2,195000.0,Residential,188000.0,0.172,N,42637
3,4,1785000.0,Commercial,1802900.0,2.155,N,42678
4,5,257575.0,Residential,210900.0,0.201,N,45228
...,...,...,...,...,...,...,...
179710,184184,90000.0,Residential,64500.0,4000.000,N,47975
179711,184185,78000.0,Residential,46700.0,2000.000,N,47991
179712,184186,11250.0,Residential,67200.0,2700.000,N,47993
179713,184187,200000.0,Residential,9200.0,1564.000,N,47993


In [119]:
#One Hot encoding for non Numerical 
Property_df = pd.get_dummies(data = Property_df, columns=['Parcel_Vacancy','Class_Code_Translated','Property_Zip'])
Property_df


,index,Property_Sales_Price,Total_Assessed_Value,Parcel_Acreage,Parcel_Vacancy_N,Parcel_Vacancy_Y,Class_Code_Translated_Agricultural,Class_Code_Translated_Commercial,Class_Code_Translated_Exempt,Class_Code_Translated_Industrial,...,Property_Zip_47989,Property_Zip_47990,Property_Zip_47991,Property_Zip_47992,Property_Zip_47993,Property_Zip_47994,Property_Zip_47995,Property_Zip_47997,Property_Zip_48448,Property_Zip_`4790
0,0,179900.0,182300.0,0.169,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,1,48000.0,93500.0,0.134,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,2,195000.0,188000.0,0.172,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,4,1785000.0,1802900.0,2.155,1,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
4,5,257575.0,210900.0,0.201,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
179710,184184,90000.0,64500.0,4000.000,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
179711,184185,78000.0,46700.0,2000.000,1,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,0
179712,184186,11250.0,67200.0,2700.000,1,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0
179713,184187,200000.0,9200.0,1564.000,1,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0


In [120]:
df = Property_df.loc[:, Property_df.columns != 'Class_Code_Translated_Locally Assessed']
df

,index,Property_Sales_Price,Total_Assessed_Value,Parcel_Acreage,Parcel_Vacancy_N,Parcel_Vacancy_Y,Class_Code_Translated_Agricultural,Class_Code_Translated_Commercial,Class_Code_Translated_Exempt,Class_Code_Translated_Industrial,...,Property_Zip_47989,Property_Zip_47990,Property_Zip_47991,Property_Zip_47992,Property_Zip_47993,Property_Zip_47994,Property_Zip_47995,Property_Zip_47997,Property_Zip_48448,Property_Zip_`4790
0,0,179900.0,182300.0,0.169,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,1,48000.0,93500.0,0.134,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,2,195000.0,188000.0,0.172,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,4,1785000.0,1802900.0,2.155,1,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
4,5,257575.0,210900.0,0.201,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
179710,184184,90000.0,64500.0,4000.000,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
179711,184185,78000.0,46700.0,2000.000,1,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,0
179712,184186,11250.0,67200.0,2700.000,1,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0
179713,184187,200000.0,9200.0,1564.000,1,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0


In [121]:
df = Property_df.loc[:, Property_df.columns != 'Class_Code_Translated_Mineral']
df

,index,Property_Sales_Price,Total_Assessed_Value,Parcel_Acreage,Parcel_Vacancy_N,Parcel_Vacancy_Y,Class_Code_Translated_Agricultural,Class_Code_Translated_Commercial,Class_Code_Translated_Exempt,Class_Code_Translated_Industrial,...,Property_Zip_47989,Property_Zip_47990,Property_Zip_47991,Property_Zip_47992,Property_Zip_47993,Property_Zip_47994,Property_Zip_47995,Property_Zip_47997,Property_Zip_48448,Property_Zip_`4790
0,0,179900.0,182300.0,0.169,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,1,48000.0,93500.0,0.134,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,2,195000.0,188000.0,0.172,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,4,1785000.0,1802900.0,2.155,1,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
4,5,257575.0,210900.0,0.201,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
179710,184184,90000.0,64500.0,4000.000,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
179711,184185,78000.0,46700.0,2000.000,1,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,0
179712,184186,11250.0,67200.0,2700.000,1,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0
179713,184187,200000.0,9200.0,1564.000,1,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0


In [122]:
#Git Rid of Index
df.drop(['index'],axis=1,inplace=True)



C:\Users\coset\Anaconda3\lib\site-packages\pandas\core\frame.py:3990: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().drop(


In [123]:
df = df.loc[df['Total_Assessed_Value']>0,:]
df = df.loc[df['Parcel_Acreage']>0,:]
#len(new_df)


## Model Build

In [124]:

#Seperate X and Y
y_data = df['Property_Sales_Price']

x_data = df.loc[:, df.columns != 'Property_Sales_Price']

In [125]:
    data = df.loc[:, df.columns !='Property_Sales_Price'] 
    print( df['Property_Sales_Price'], data, data.columns )


0          179900.0
1           48000.0
2          195000.0
3         1785000.0
4          257575.0
            ...    
179710      90000.0
179711      78000.0
179712      11250.0
179713     200000.0
179714     184000.0
Name: Property_Sales_Price, Length: 171928, dtype: float64         Total_Assessed_Value  Parcel_Acreage  Parcel_Vacancy_N  \
0                   182300.0           0.169                 1   
1                    93500.0           0.134                 1   
2                   188000.0           0.172                 1   
3                  1802900.0           2.155                 1   
4                   210900.0           0.201                 1   
...                      ...             ...               ...   
179710               64500.0        4000.000                 1   
179711               46700.0        2000.000                 1   
179712               67200.0        2700.000                 1   
179713                9200.0        1564.000                 

In [126]:
feature_names = data.columns

In [127]:
data.shape

(171928, 987)

In [128]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 171928 entries, 0 to 179714
Columns: 987 entries, Total_Assessed_Value to Property_Zip_`4790
dtypes: float64(2), uint8(985)
memory usage: 165.4 MB


In [129]:
data.describe()

,Total_Assessed_Value,Parcel_Acreage,Parcel_Vacancy_N,Parcel_Vacancy_Y,Class_Code_Translated_Agricultural,Class_Code_Translated_Commercial,Class_Code_Translated_Exempt,Class_Code_Translated_Industrial,Class_Code_Translated_Locally Assessed,Class_Code_Translated_Residential,...,Property_Zip_47989,Property_Zip_47990,Property_Zip_47991,Property_Zip_47992,Property_Zip_47993,Property_Zip_47994,Property_Zip_47995,Property_Zip_47997,Property_Zip_48448,Property_Zip_`4790
count,1.719280e+05,171928.000000,171928.000000,171928.000000,171928.000000,171928.000000,171928.000000,171928.000000,171928.000000,171928.000000,...,171928.000000,171928.000000,171928.000000,171928.000000,171928.000000,171928.000000,171928.000000,171928.000000,171928.0,171928.000000
mean,1.471466e+05,2867.616932,0.894130,0.105870,0.060037,0.036736,0.005491,0.004793,0.000326,0.892606,...,0.000023,0.000012,0.000087,0.000192,0.000558,0.000012,0.000297,0.000012,0.0,0.000006
std,3.574944e+05,21266.499256,0.307672,0.307672,0.237556,0.188114,0.073896,0.069063,0.018045,0.309614,...,0.004823,0.003411,0.009340,0.013853,0.023623,0.003411,0.017221,0.003411,0.0,0.002412
min,9.000000e+01,0.001000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000
25%,5.060000e+04,0.179000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000
50%,1.090000e+05,0.345000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000
75%,1.827000e+05,4.725250,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000
max,3.526210e+07,417500.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,...,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,0.0,1.000000


In [130]:

#to see pairwise relationships on our dataset we will check pairplot from seaborn library
seaborn.pairplot(data)

KeyboardInterrupt: 

Error in callback <function flush_figures at 0x000002527AB20670> (for post_execute):


KeyboardInterrupt: 

In [ ]:
#create correlation matrix
correlations = data.corr()
indx=correlations.index

#plot this correlation for clear visualisation
plt.figure(figsize=(26,22))
#annot = True , dsiplays text over the cells.
#cmap = "YlGnBu" is nothing but adjustment of colors for our heatmap
seaborn.heatmap(data[indx].corr(),annot=True,cmap="YlGnBu")
#amount of darkness shows how our features are correalated with each other

In [150]:

def split_data_train_model(labels, data):
    labels = labels.values.reshape(-1,1)
    Xscaler = StandardScaler()
    Xscaler = Xscaler.fit(data)
    Xscaled = Xscaler.transform(data)
    Yscaler = StandardScaler()
    Yscaler = Yscaler.fit(labels)
    Yscaled = Yscaler.transform(labels)
    # 20% examples in test data
    train, test, train_labels, test_labels = train_test_split(Xscaled,
                                                              Yscaled,
                                                              test_size=0.25,
                                                              random_state=RF_SEED)
    print(train)
    print(train_labels)
# training data fit
    regressor = RandomForestRegressor(n_estimators = 100, random_state = RF_SEED, max_depth=5)
    regressor = regressor.fit(train, train_labels)
    regressor.score(train, train_labels)
    return test, test_labels, regressor, Xscaler, Yscaler 

In [151]:
x_test, y_test, regressor, Xscaler, Yscaler = split_data_train_model(y_data, x_data)
 
predictions_test = regressor.predict(x_test)
print(predictions_test)
print(y_test)


[[-1.34398456e-01 -1.34448797e-01  3.44100966e-01 ... -3.41070547e-03
   0.00000000e+00 -2.41172596e-03]
 [ 2.13579987e-01 -1.34830996e-01  3.44100966e-01 ... -3.41070547e-03
   0.00000000e+00 -2.41172596e-03]
 [-9.07612880e-02 -1.34835792e-01  3.44100966e-01 ... -3.41070547e-03
   0.00000000e+00 -2.41172596e-03]
 ...
 [-2.45169729e-01 -1.34835886e-01  3.44100966e-01 ... -3.41070547e-03
   0.00000000e+00 -2.41172596e-03]
 [-2.77338154e-01 -1.34820369e-01  3.44100966e-01 ... -3.41070547e-03
   0.00000000e+00 -2.41172596e-03]
 [-3.67130020e-01  8.17585850e+00 -2.90612378e+00 ... -3.41070547e-03
   0.00000000e+00 -2.41172596e-03]]
[[-0.06357317]
 [ 0.07208747]
 [-0.08545911]
 ...
 [-0.18346306]
 [-0.11748339]
 [-0.27680015]]


<ipython-input-150-cfd0aa8e37a9>:18: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  regressor = regressor.fit(train, train_labels)


[ 0.21192221 -0.05888015 -0.17687073 ... -0.07691865  0.2201111
  0.22367264]
[[ 0.31862613]
 [-0.07564262]
 [-0.32668722]
 ...
 [-0.2059927 ]
 [-0.02076685]
 [ 0.25586498]]


In [152]:
# find the correlation between real answer and prediction
#correlation_test = round(pearsonr(predictions_test, x_test_labels)[0], 5)
#correlation_train = round(pearsonr(predictions_train, x_test_labels)[0], 5)
#print(correlation_test)
#print(correlation_train)

In [153]:
features_importance = regressor.feature_importances_
 
print("Feature ranking:")
for i, data_class in enumerate(feature_names):
    print("{}. {} ({})".format(i + 1, data_class, features_importance[i]))


Feature ranking:
1. Total_Assessed_Value (0.7665496560144001)
2. Parcel_Acreage (0.023041780093659807)
3. Parcel_Vacancy_N (0.008173844484498453)
4. Parcel_Vacancy_Y (0.005369347345920151)
5. Class_Code_Translated_Agricultural (0.0)
6. Class_Code_Translated_Commercial (0.0033615097438252084)
7. Class_Code_Translated_Exempt (0.007561992911196941)
8. Class_Code_Translated_Industrial (0.007384251437174532)
9. Class_Code_Translated_Locally Assessed (0.0)
10. Class_Code_Translated_Residential (0.01253094170800235)
11. Property_Zip_41750 (0.0)
12. Property_Zip_42601 (0.0)
13. Property_Zip_42637 (0.0)
14. Property_Zip_42678 (0.0)
15. Property_Zip_45228 (0.0)
16. Property_Zip_46001 (0.0)
17. Property_Zip_46011 (0.0)
18. Property_Zip_46012 (0.0)
19. Property_Zip_46013 (0.0)
20. Property_Zip_46014 (0.0)
21. Property_Zip_46015 (0.0)
22. Property_Zip_46016 (0.00034474074901412645)
23. Property_Zip_46017 (0.0)
24. Property_Zip_46018 (0.0)
25. Property_Zip_46030 (0.0)
26. Property_Zip_46031 (0.0)
27

In [154]:
print(regressor.score(x_test, y_test))

0.6357135705501051


In [26]:
pickle.dump(regressor, open('model.pkl','wb'))

In [26]:
pickle.dump(Xscaler, open('Xscaler.pkl','wb'))

In [27]:
pickle.dump(Yscaler, open('Yscaler.pkl','wb'))